In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale

import sklearn.metrics as sm
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
from sklearn import datasets
from sklearn.preprocessing import Imputer
import matplotlib.patches as mpatches

import seaborn as sns

print(__doc__)

from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from matplotlib.ticker import MaxNLocator
from collections import namedtuple
matplotlib.rcParams.update({'font.size': 22})

## Read in data as csv 

In [ ]:
data=pd.read_table('/Users/gracer/Google Drive/ABCD/important_txt/4Kmeans.csv', sep=',')

## Seperate by sex

In [ ]:
dudes = data[data['sex'] >= 0]
lady_dudes = data[data['sex'] >= 1]

## Get a List of variable names

In [ ]:
var_names=list(lady_dudes.columns.values)
cols = lady_dudes.columns
print(cols)
cols[1]

## Combine variables of interest into a single matrix

In [ ]:
labels_true=lady_dudes['PDS'].values
f2=lady_dudes['pds_ht2_y'].values
f3=lady_dudes['pds_skin2_y'].values
f4=lady_dudes['pds_bdyhair_y'].values
f5=lady_dudes['pds_f4_2_y'].values
f6=lady_dudes['pds_f5_y'].values
f7=lady_dudes['interview_age'].values
f8=lady_dudes['anthroheightcalc'].values 
f9=lady_dudes['anthroweightcalc'].values
f10=lady_dudes['anthro_waist_cm'].values
f11=lady_dudes['hormone_scr_dhea_mean'].values
f12=lady_dudes['hormone_scr_hse_mean'].values
f13=lady_dudes['hormone_scr_ert_mean'].values
X=np.matrix(zip(f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13))

## Create a list of the variable names included in this analyses

In [ ]:
names=['pds_ht2_y',
'pds_skin2_y'
'pds_bdyhair_y',
'pds_f4_2_y',
'pds_f5_y',
'interview_age',
'anthroheightcalc',
'anthroweightcalc',
'anthro_waist_cm',
'hormone_scr_dhea_mean',
'hormone_scr_hse_mean',
'hormone_scr_ert_mean']

## Check the data 

In [ ]:
%matplotlib inline
plt.hist(labels_true)
y = np.bincount(labels_true.astype(int))
ii = np.nonzero(y)[0]
zip(ii,y[ii])

An issue is a low number of people in groups 3 and 4

## Possible solution, randomly sample equal numbers
Using the rule of thumb 2^m I need 8 people per cluster 
Possible combinations below

In [ ]:
import math
x=math.factorial(70)
y=math.factorial(70-20)
fact=x/y
print(fact)

## Create target variable (or the one you are comparing)

In [ ]:
target_var=pd.DataFrame(lady_dudes['PDS'].values)

## Impute missing values
This will not allow missing data, so have to impute nans

In [ ]:
imputer = Imputer()
transformed_values = imputer.fit_transform(X)
# count the number of NaN values in each column
print(np.isnan(transformed_values).sum()) 
transformed_values_scale = scale(transformed_values)
#the target variable is the last variable
trans = np.hstack((transformed_values_scale,target_var.round(decimals=0)))

In [ ]:
transformed_values.shape

## Function to randomly sample the data and perform the kmeans

In [ ]:
def bench_k_means(estimator, name, data):
        t0 = time() #time
        estimator.fit(data) #estimating the fit 
        print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
              % (name, (time() - t0), estimator.inertia_,
                 metrics.homogeneity_score(labels, estimator.labels_),
                 metrics.completeness_score(labels, estimator.labels_),
                 metrics.v_measure_score(labels, estimator.labels_),
                 metrics.adjusted_rand_score(labels, estimator.labels_),
                 metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
                 metrics.silhouette_score(data, estimator.labels_,
                                          metric='euclidean',
                                          sample_size=sample_size)))


In [ ]:
def meaner(**kwargs):
    for x in kwargs.items():
        print(x)
#     z=sum(*arg)/len(*arg)
#     return z

In [ ]:
def PCA_reducer(DATA):
    n_samples, n_features = DATA.shape
    labels = np.round(targets)
    n_digits = len(np.unique(targets))
    sample_size=n_samples
    
    PCA_results = PCA(n_components=2)
    reduced_data = PCA_results.fit_transform(DATA)
    
    # Dump components relations with features:
    print pd.DataFrame(PCA_results.components_,index = ['PC-1','PC-2'])
    plt.semilogy(PCA_results.explained_variance_ratio_, '--o')
    return (PCA_results.components_, PCA_results.explained_variance_ratio_, 
            PCA_results.explained_variance_, PCA_results.mean_) 
    


In [ ]:
def random_selct(DATA,i):
    print('start')
    dictr = {}
    #defining the levels, based on the final column add the values to the dictionary
    levels = ['lev1','lev2','lev3']
    i=1
    for lev in levels:
        if i < len(levels)+1:
            dictr[lev] = DATA[np.where(DATA[:,-1] == i)]
            i=i+1
    
    rand_dict={}
    target_dict={}
    ind_dict={}
    for key, value in dictr.iteritems():
        #shuffle the data's index
        ind = np.random.permutation(value.shape[0])#random index
        #get the first 20 subjects indexes
        training_idx = ind[:20]#get 20 subjects indexes
        #get the first 20 subjects
        training = value[training_idx,:]#select 20 subjects from the value in the dictionary
        #saving the true labels
        labels_true = training[:,-1] #get the labels from the value in the dictiornary last column
        target_dict[key] = labels_true #add targets to dictionary
        rand_dict[key] = training #match the randomized data to the ind_dict by key 
        ind_dict[key] = training_idx #add the indexes to the dictionary
    
    '''
    combine all the dictionaries we have created thus far. 
    data, index, and targets
    '''
    
    #combine the randomized data by the actual level
    data=np.vstack((rand_dict['lev1'],rand_dict['lev2'],rand_dict['lev3']))
    
    ###################################################
    data=np.delete(data,12,1)# remove the label column
    ###################################################
    
    
    #combine the true labels into targets
    targets=np.hstack((target_dict['lev1'],target_dict['lev2'],target_dict['lev3']))
    #combine the indexes into one
    indexes=np.hstack((ind_dict['lev1'],ind_dict['lev2'],ind_dict['lev3']))
    
    '''
    defining the parameters for the k means and the PCA
    '''
    
    n_samples, n_features = data.shape
    labels = np.round(targets)
    n_digits = len(np.unique(targets))
    sample_size=n_samples

    '''
    defining the parameters for the k means
    '''    
    kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=300)
    
    ###########################################
    #            rebuild to combine data      #
    ###########################################
    comb_data = np.column_stack((data, kmeans.fit_predict(data)))#makes first column the index value
    full_data = np.column_stack((indexes, comb_data))#makes first column the index value
    comb_dictr = {}
    '''
    seperating the data based on the fit predict value (found right above)
    '''
    i=0
    for lev in levels:
        if i < len(levels)+1:
            comb_dictr[lev] = comb_data[np.where(comb_data[:,-1] == i)]
            i=i+1
    '''
    defining the parameters for the PCA and running the PCA 
    '''
    ###########################################
    #            PCA                          #
    ###########################################
    PCA_dict={}
    PCA_data={}
    PCA_final = {}

    for key, value in comb_dictr.iteritems():
        PCA_dict[key] = PCA(n_components=2)
        #make sure the last column which is the predicted label isn't included
#         print(value[:,0:12])
        PCA_data[key] = PCA_dict[key].fit_transform(value[:,0:12])
        #make sure the last column which is the predicted label isn't included
        ID = np.identity(value[:,0:12].shape[1])  # identity matrix
        coef = PCA_dict[key].transform(ID)
        PCA_final[key] = coef
        
        plt.semilogy(PCA_dict[key].explained_variance_ratio_, '--o', label = '%s'%key)
    plt.legend()        
        
    

    ####THINGS TO SAVE#############################################################################        
    kmeans.fit(data)
    y_kmeans = kmeans.predict(data)
    clusters = kmeans.fit_predict(data)
    cluster_space = kmeans.fit_transform(data)
    
    
    
    centroids = kmeans.cluster_centers_   
    
    Components = np.vstack((PCA_dict['lev1'].components_,
                            PCA_dict['lev2'].components_,PCA_dict['lev3'].components_))
    
#     Var_ratio = np.vstack((PCA_final['lev1'],
#                              PCA_final['lev2'],PCA_final['lev3']))
    Var_ratio = PCA_final
    
    lev1_VarRatio = np.vstack((PCA_final['lev1']))
    print(lev1_VarRatio)
    lev2_VarRatio = np.vstack((PCA_final['lev2']))
    lev3_VarRatio = np.vstack((PCA_final['lev3']))
    
#     Var = np.vstack((PCA_dict['lev1'].explained_variance_,
#                            PCA_dict['lev2'].explained_variance_,PCA_dict['lev3'].explained_variance_))            
    Mean = np.vstack((PCA_dict['lev1'].mean_,
                           PCA_dict['lev2'].mean_,PCA_dict['lev3'].mean_))                          
    ind_dict = np.hstack((ind_dict['lev1'],
                            ind_dict['lev2'],ind_dict['lev3']))
    
    ########################################################################################################
    return (clusters, cluster_space, centroids, 
            targets, ind_dict,
           Components, Var_ratio, 
           full_data,
           lev1_VarRatio,
           lev2_VarRatio,
           lev3_VarRatio)

# 0clusters
# 1cluster_space, 
# 2centroids, 
# 3targets, 
# 4ind_dict,
# 5Components,
# 6Var_ratio, 
# 7full_data,
# 8lev1_VarRatio,
# 9lev2_VarRatio,
# 10lev3_VarRatio

In [ ]:
def main(i):
    names = ['pds_ht2_y',
'pds_skin2_y',
'pds_bdyhair_y',
'pds_f4_2_y',
'pds_f5_y',
'interview_age',
'anthroheightcalc', 
'anthroweightcalc',
'anthro_waist_cm',
'hormone_scr_dhea_mean',
'hormone_scr_hse_mean',
'hormone_scr_ert_mean', 'labels_pred']
    
#     Y_means = []
    Clusters = []
    Cluster_space = []
    Centroids = []
    
    Comp = []
    Var_ratio = []
#     Var = []
#     Mean = []
    Targets = []
    Indexes = []
    Full_data = []
    Lev1_VarRatio = []
    Lev2_VarRatio = []
    Lev3_VarRatio = []
    
    for x in range(0,i):
        Q = random_selct(trans,i)
#         Y_means.append(Q[0])
        Clusters.append(Q[0])
        Cluster_space.append(Q[1])
        Centroids.append(Q[2])
        Targets.append(Q[3])
        Indexes.append(Q[4])
#         Var.append(Q[8])
        Comp.append(Q[5])
        Var_ratio.append(Q[6])
        Full_data.append(Q[7])
        Lev1_VarRatio.append(Q[8].transpose())
        print(Lev1_VarRatio)
        Lev2_VarRatio.append(Q[9].transpose())
        Lev3_VarRatio.append(Q[10].transpose())
    
    All_lev1Ratio = np.vstack(Lev1_VarRatio)
    print(All_lev1Ratio)
    All_lev2Ratio = np.vstack(Lev2_VarRatio)
    All_lev3Ratio = np.vstack(Lev3_VarRatio)
    
    All_clust = np.hstack(Clusters)
    All_clust_space = np.vstack(Cluster_space)
#     All_means = np.hstack(Y_means)
    All_labels = np.hstack(Targets)
    All_index = np.hstack(Indexes)
    All_comp = np.vstack(Comp)
#    All_var_ratio = np.vstack(Var_ratio)
    All_var_ratio = Var_ratio
#     All_var = np.hstack(Var)
    All_data = np.vstack(Full_data)
    
#     All_data=pd.DataFrame(data=All_data[1:,1:],index=All_data[1:,0], columns= names)
#    New_names = ['mean','true_labels']
#    New_data=np.vstack((All_index, All_labels))
#     New_data=New_data.transpose()
#     New_data=pd.DataFrame(data=New_data[1:,1:],index=New_data[1:,0],columns = New_names)
    
    return (All_clust ,All_clust_space,  
            All_labels , All_index, All_comp, All_var_ratio, 
            All_data, names, 
           All_lev1Ratio, All_lev2Ratio, All_lev3Ratio)

In [ ]:
All_clust ,All_clust_space,All_labels , All_index, All_comp, All_var_ratio,All_data, names, All_lev1Ratio, All_lev2Ratio, All_lev3Ratio = main(50)

In [ ]:
#transformed_values
# comb_data = np.column_stack((indexes, transformed_values))#makes first column the index value
# comb_data = np.column_stack((comb_data, kmeans.fit_predict(data)))
# pd.DataFrame(data=transformed[1:,1:],index=All_data[1:,0], columns= names)

In [ ]:
All_lev1Ratio.shape

## Taking the variance from each K mean cluster and PCA, using PCA to find the most important factors over all

In [ ]:
All_Ratios = {'Lev1': All_lev1Ratio, 'Lev2': All_lev2Ratio, 'Lev3':All_lev3Ratio}
Coefs = {}
index = np.arange(12)
for key, value in All_Ratios.iteritems(): 
    pca = PCA(n_components=2)
    pca.fit_transform(value)
    i = np.identity(value.shape[1])  # identity matrix
    Coefs[key] = pca.transform(i)
    print(key)
    print pd.DataFrame(pca.components_,columns=names[0:12],index = ['PC-1','PC-2'])
    print('\n')
    
    n_groups = 12

    means_CP1 = np.absolute(pca.components_[0,:])
    std_CP1 = np.matrix.std(np.matrix(np.absolute(pca.components_[0,:])))

    means_CP2 = np.absolute(pca.components_[1,:])
    std_CP2 = np.matrix.std(np.matrix(np.absolute(pca.components_[1,:])))

    fig, ax = plt.subplots(figsize=(45, 20))
    bar_width = 0.35

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = ax.bar(index, means_CP1, bar_width,
                    alpha=opacity, color='b',
                    yerr=std_CP1, error_kw=error_config,
                    label='Component 1')

    rects2 = ax.bar(index + bar_width, means_CP2, bar_width,
                    alpha=opacity, color='r',
                    yerr=std_CP2, error_kw=error_config,
                    label='Component 2')
    #plt.xlabel( fontsize=18)
    ax.set_xlabel('Factors', fontsize = 50)
    ax.set_ylabel('Coefficient Weight (Beta)', fontsize = 50)
    ax.set_title('Which factors are the most meaning full for %s cluster'%key, fontsize = 50)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(names[0:12], fontsize = 28, rotation = 25)
    ax.legend()
    ax.tick_params(labelsize = 38)

    fig=plt.figure(figsize=(100, 16), dpi= 80, facecolor='w', edgecolor='k')
    fig.tight_layout()
    plt.show()

## A Nice Plot example

In [ ]:
# # print(All_data)
# # df2 = pd.DataFrame(group.describe().rename(columns={'interview_age':labels_pred}).squeeze()
# #                          for name, group in All_data.groupby('labels_pred'))

# # print(df2)
# subs=list(All_data.index.values)
# len(subs)
# All_data.interview_age.describe()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import figure
# from matplotlib.ticker import MaxNLocator
# from collections import namedtuple
# matplotlib.rcParams.update({'font.size': 22})


# n_groups = 12

# means_CP1 = np.absolute(pca.components_[0,:])
# std_CP1 = np.matrix.std(np.matrix(np.absolute(pca.components_[0,:])))

# means_CP2 = np.absolute(pca.components_[1,:])
# std_CP2 = np.matrix.std(np.matrix(np.absolute(pca.components_[1,:])))

# fig, ax = plt.subplots(figsize=(45, 20))



# index = np.arange(n_groups)
# bar_width = 0.35

# opacity = 0.4
# error_config = {'ecolor': '0.3'}

# rects1 = ax.bar(index, means_CP1, bar_width,
#                 alpha=opacity, color='b',
#                 yerr=std_CP1, error_kw=error_config,
#                 label='Component 1')

# rects2 = ax.bar(index + bar_width, means_CP2, bar_width,
#                 alpha=opacity, color='r',
#                 yerr=std_CP2, error_kw=error_config,
#                 label='Component 2')
# #plt.xlabel( fontsize=18)
# ax.set_xlabel('Factors', fontsize = 50)
# ax.set_ylabel('Coefficient Weight (Beta)', fontsize = 50)
# ax.set_title('Which factors are the most meaning full for Lev 1 cluster', fontsize = 50)
# ax.set_xticks(index + bar_width / 2)
# ax.set_xticklabels(names[0:12], fontsize = 28, rotation = 25)
# ax.legend()
# ax.tick_params(labelsize = 38)

# fig=plt.figure(figsize=(100, 16), dpi= 80, facecolor='w', edgecolor='k')
# fig.tight_layout()
# plt.show()

* All_clust (180,)<- length of the subjects over all runs
* All_clust_space(180,)<-length of the subjects over all runs
* All_means(180,)<-length of the subjects
* All_labels(180,) <- length of subjects True Labels
* All_index(180,)<-length of subjects Index of the original data
* All_comp(18, 12)<- rows are the the PCA components, times the number of the levels, and the number of runs. cols are the factors
* All_var_ratio(108, 2)<-rows are the number of runs, times the number of factors, times the number of levels
* All_var(3, 6)<- rows are the number of runs, the columns are the PCA componenets times the number of levels
* All_mean(9, 12)<- rows are the number of runs by the number of levels the columns are the number of factors
* All_data(180, 14)<-rows are the number of subjects, the columns are number of factors plus the index, plus the components

* components_ : array, shape (n_components, n_features)

* explained_variance_ratio_ : array, shape (n_components,)

*  singular_values_ : array, shape (n_components,)

*  mean_ : array, shape (n_features,)

*  n_components_ : int

*  noise_variance_ : float